<img src="images/GEMS long.png" width=600 alt="GEMS Logo" title="GEMS" />

# **Spatial Regression**

**Instructors:**  
Yuan Chai (chaix026@umn.edu)  
Ali Joglekar (joglekar@umn.edu)  

---

## Method of Delivery
- Workshop text, images and R code are all contained within a Jupyter Notebook hosted on the GEMS Informatics Platform. You do not need to have R or RStudio installed on your machine to participate
- A recording of the workshop will be posted on Canvas
- Please download any desired materials, as we cannot guarantee access to the Canvas course beyond 3 months.  

---  

## Module Outline: 
- [Raster data in R](#Raster)  
  - [Exercise 1](#Ex1)
- [Raster manipulation](#Manipulation) 
- [Spatial operations](#SpOp)
  - [Exercise 2](#Ex2)
- [Geometry operations](#GeomOp)  
  - [Exercise 3](#Ex3)
- [Raster-Vector interactions](#RasVec)
  - [Exercise 4](#Ex4)

---  


### **Load R Libraries**

In [ ]:
# Install and load packages needed for this workshop

packages_to_load <- c("raster", "sf", "tmap", "data.table")

for ( package in packages_to_load ) {
    # Check if package is already installed, if not, install and load the package
    if (!require(package, character.only=T, quietly=T, warn.conflicts=F)) {
        install.packages(package)
        suppressPackageStartupMessages(library(package, character.only=T, quietly=T, warn.conflicts=F))
    }
}

# Set plot options
options(repr.plot.width = 8, repr.plot.height = 4)


<details>
    
<summary><span style='color:Green'> Note: For your own work, once packages are installed, it is recommended to just use library() to load your packages  </span></summary>


```
# Spatial data libraries
library(raster)
library(sf)

# Plotting libraries
library(tmap)

# General data analysis libraries
library(data.table)
```
----
          
</details>


### Data

Your expercises will draw on datasets from the Spatial Production Allocation Model (SPAM) and Minnesota Geospatial Commons, which have been downloaded, cleaned, transformed and saved in the directory `./data/MN` for this workshop.

### Attribution

The following material draws heavily from Robin Lovelace's [Geocomputation with R](https://geocompr.robinlovelace.net/index.html) textbook.

<a id="Raster"></a>
## **Spatial Dependence**

When dealing with space, one must bear in mind Tobler’s first law of geography  

  > **“Everything is related to everything else, but close things are more related than things that are far apart”**  
  > --Tobler (1979)  
  
In this section we’ll focus on the specification of spatial dependence, on specification tests to detect spatial dependence in regressions models and on basic regression models that incorporate spatial dependence.



<a id="Ex1"></a>
### **<span style='color:Green'> Exercise 1: Read in raster files and create raster stack </span>**   

In this exercise, we will first read in raster files (in `.tif` format) which contains the maize harvested area and production in Minnesota and then create a raster stack.  


**Step 1:** Use the `raster()` function to read in raster files from the `'./data/MN/spam/` directory.    

>file `./data/MN/spam/spam_H_MAIZ_A_mn.tif` is the Harvested Area (in hecacres) for Maize in Minnesota  
>file `./data/MN/spam/spam_P_MAIZ_A_mn.tif` is the Production (in metric tons) for Maize in Minnesota  

Use various summarizing functions (e.g., `extent()`, `res()`, `crs()`) to check the two raster files having the same extent and resolution

In [ ]:
# Read in raster files
spam_H_MAIZ_A_mn <- raster("./data/MN/spam/spam_H_MAIZ_A_mn.tif")
spam_P_MAIZ_A_mn <- raster("./data/MN/spam/spam_P_MAIZ_A_mn.tif")

In [ ]:
# Check for extent, resolution, and crs


<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```
# Read in raster files
spam_H_MAIZ_A_mn <- raster("./data/MN/spam/spam_H_MAIZ_A_mn.tif")
spam_P_MAIZ_A_mn <- raster("./data/MN/spam/spam_P_MAIZ_A_mn.tif")

# Check for extent, resolution, and crs
extent(spam_H_MAIZ_A_mn)
extent(spam_P_MAIZ_A_mn)

res(spam_H_MAIZ_A_mn)
res(spam_P_MAIZ_A_mn)

crs(spam_H_MAIZ_A_mn)
crs(spam_P_MAIZ_A_mn)
```
----
          
</details>

<a id="Ex5"></a>
### **<span style='color:Green'> At-home Exercise: Learn `terra`  </span>**   

Now that you are familiar with raster data operations in R, it is recommended that you learn `terra` on your own and start use R for raster data analysis for your own work.

Some useful resources to get you started  
>**[The `terra` package](https://rspatial.org/terra/pkg/index.html)**  

>**[A comparison of terra and raster packages](https://www.r-bloggers.com/2021/05/a-comparison-of-terra-and-raster-packages/)**  

>**[Spatial Data Science with R and “terra”](https://rspatial.org/terra/index.html)**

